# Morphological Inflection in Navajo

## Problem Description

Morphological inflection is a task in computational linguistics wherein the correct form of a word has to be generated from a lemma (base form) and a target morphosyntactic specification, e.g.


```
alzhish  +  V;IND;PFV;NOM(2,SG) -> íínílzhiizh
```


where *alzhish* is the Navajo word for 'dance', and *íínílzhiizh* is the second [2] person , singular [SG], nominative [NOM] form of the verb [V] in the indicative [IND] mood and perfect [PFV] aspect. You can find out more about the annotation schema used for the morphosyntactic specification of the target forms [here](https://unimorph.github.io/doc/unimorph-schema.pdf).

This task finds application in the construction of resources for language acquisition, in statistical machine translation, and in the computational study of language.

The difficulty of the task for a given language depends on the size and complexity of the inflectional paradigm of the language. A large paradigm, which distinguishes between many morphosyntactic properties (number, gender, case, aspect, etc.) and many values for these properties, requires extensive machine learning from sizeable data. While inflectional paragims tend to be highly regular, they can vary in complexity, with many factors guiding which exact morpheme should attach to a given root to mark a specific morphosyntactic property. And irregular forms can often be found too, which cannot be predicted and have to be memorized instead.

Here, we ask you to train a machine learning model to perform morphological inflection in Navajo.

## Your Task

The code below is a near reimplementation of the approach to moprhological inflection presented in [Wu et al.](https://aclanthology.org/2021.eacl-main.163.pdf). The reimplementation uses high-level API from the *transformers* Python library. High-level APIs are convenient as they save us a lot of code-writing, but they also obscure certain aspects of the implementation.

The approach of Wu and colleagues achieves a score of 52.1% for Navajo (as reported [here]()), while the score of the model below lingers about 3 percentage points behind. Study the approach of Wu and colleagues as described in their article to find out what is missing from the model below and make the necessary changes to reproduce their result.   

Notice that due to the stochasticity of the model initialization and training, slight deviations from the expected score are admissible (within 0.5 percentage points).

### Deliverables:
* code
* model accuracy on the test set
* a report of up to 300 words describing the changes made to the model architecture, data processing, training procedure etc.
    * If you do not succeed in reproducing the expected score, comment on what the reason may be.


## Technical Specifications

* All team solutions should be submitted as a modified and compiled copy of this base notebook.
* Do not change cells starting with the `###DO NOT CHANGE THIS CELL###` comment.
* You can use any platform to carry out the development of your model, but for the final submission you have to integrate your solution back into this notebook, adding code to install all dependencies and making sure that your model takes no longer than 8 hours to train.




In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install -U evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
###DO NOT CHANGE THIS CELL###
"""
Download the train and dev data
"""

import requests

def download_data(path):
        response = requests.get(path)
        return response.text.strip().split('\n')

train_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.trn'
dev_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.dev'

data = {}
data['train'] = download_data(train_data_path)
data['dev'] = download_data(dev_data_path)

print('Downloaded {} training data samples and {} dev samples.'.format(len(data['train']), len(data['dev'])))
print('Raw data sample:', data['train'][54])

Downloaded 10000 training data samples and 1000 dev samples.
Raw data sample: adiłhash	V;IND;PFV;NOM(3,GRPL)	daʼdiłhash


# Task 1
Modify the code below to achieve a full reimplementation of the approach of Wu and colleagues.

In [2]:
"""
DATA PREPROCESSING
"""

import re
import regex

def parse_tag(tag):
    tag = re.sub(r"\)|\(|,|;", ' ', tag).split()
    return ''.join(['<{}>'.format(t) for t in tag])

def preprocess_data(raw_data):
        preprocessed_data = []
        for line in raw_data:
          lemma, tag, target = line.split('\t')
          preprocessed_data.append(('{} {}'.format(lemma, parse_tag(tag)),target))
        return preprocessed_data

data['train'] = preprocess_data(data['train'])
data['dev'] = preprocess_data(data['dev'])

print('Preprocessed data sample:', data['train'][54])

chars = set(list(''.join([''.join([d[0].split()[0], d[1]]) for d in data['train']])))
char2id = { char: i for i, char in enumerate(chars)}
tags = list(set(sum([regex.findall(r"<[A-Za-z0-9]*>",d[0]) for d in data['train']], [])))

Preprocessed data sample: ('adiłhash <V><IND><PFV><NOM><3><GRPL>', 'daʼdiłhash')


In [3]:
"""
TOKENIZER
Input and output word forms are processed one character at a time, while morphosyntactic features are treated as special atomic tokens.
"""

import json
from pathlib import Path
from typing import Optional, Tuple, Dict, List

from transformers import PreTrainedTokenizer
from transformers.tokenization_utils import AddedToken

import warnings

class CustomTokenizer(PreTrainedTokenizer):

    model_input_names = ["input_ids", "attention_mask"]

    def __init__(
        self,
        vocab: Dict[str, int],
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        **kwargs,
    ) -> None:
        # Add extra_ids to the special token list

        self.__token_ids = vocab
        self.__id_tokens: Dict[int, str] = {value: key for key, value in vocab.items()}

        pad_token = AddedToken(pad_token, lstrip=False, rstrip=False) if isinstance(pad_token, str) else pad_token
        bos_token = AddedToken(bos_token, lstrip=False, rstrip=False) if isinstance(bos_token, str) else bos_token
        eos_token = AddedToken(eos_token, lstrip=False, rstrip=False) if isinstance(eos_token, str) else eos_token
        unk_token = AddedToken(unk_token, lstrip=False, rstrip=False) if isinstance(unk_token, str) else unk_token
        self._added_tokens_decoder = {0: pad_token, 1: bos_token, 2: eos_token, 3: unk_token}
        self.offset = len(self._added_tokens_decoder)

        super().__init__(
            bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            **kwargs,
        )

    @property
    def vocab_size(self) -> int:
        return len(self.__token_ids)

    def get_vocab(self) -> Dict[str, int]:
        vocab = {self.convert_ids_to_tokens(i): i for i in range(self.vocab_size + self.offset)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _add_eos(self, token_ids: List[int]) -> List[int]:
        return token_ids + [self.eos_token_id]

    def create_token_type_ids_from_sequences(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        eos = [self.eos_token_id]

        if token_ids_1 is None:
            return len(token_ids_0 + eos) * [0]
        return len(token_ids_0 + eos + token_ids_1 + eos) * [0]

    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        token_ids_0 = self._add_eos(token_ids_0)
        if token_ids_1 is None:
            return token_ids_0
        else:
            token_ids_1 = self._add_eos(token_ids_1)
            return token_ids_0 + token_ids_1

    def _tokenize(self, text: str, **kwargs):
        return list(text)

    def _convert_token_to_id(self, token: str) -> int:
        return self.__token_ids[token]+self.offset if token in self.__token_ids else self.unk_token_id

    def _convert_id_to_token(self, index: int) -> str:
        return self.__id_tokens[index-self.offset] if index-self.offset in self.__id_tokens else self.unk_token

    def convert_tokens_to_string(self, tokens):
        return "".join(tokens)

    def save_vocabulary(self, save_directory: str,
                        filename_prefix: Optional[str] = None) -> Tuple[str]:
        if filename_prefix is None:
            filename_prefix = ''
        vocab_path = Path(save_directory, filename_prefix + 'vocab.json')
        json.dump(self.__token_ids, open(vocab_path, 'w'))
        return str(vocab_path),

tokenizer = CustomTokenizer(char2id, additional_special_tokens=tags, max_len=100)
print('Tokenization example:', tokenizer.tokenize(data['train'][54][0]))

Tokenization example: ['a', 'd', 'i', 'ł', 'h', 'a', 's', 'h', ' ', '<V>', '<IND>', '<PFV>', '<NOM>', '<3>', '<GRPL>']


In [4]:
import torch.nn as nn
import torch

class BartLearnedPositionalEmbedding(nn.Embedding):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        self.offset = 2
        super().__init__(num_embeddings + self.offset, embedding_dim)

    def forward(self, input_ids: torch.Tensor, past_key_values_length: int = 0):
        bsz, seq_len = input_ids.shape[:2]
        device = input_ids.device

        mask_24 = (input_ids == 20)
        idx_24 = torch.where(mask_24.any(dim=1), mask_24.float().argmax(dim=1), torch.full((bsz,), seq_len-1, device=device, dtype=torch.long))
        positions = torch.arange(1, seq_len+1, dtype=torch.long, device=device).unsqueeze(0).expand(bsz, -1)
        mask_after_24 = torch.arange(seq_len, device=device).unsqueeze(0).expand(bsz, -1) > idx_24.unsqueeze(1)

        positions = positions.masked_fill(mask_after_24, 0)

        return super().forward(positions + self.offset)

In [23]:
"""
MODEL
The model is based on the t5 model architecture: a transformer-based model with an encoder
and a decoder, trained to take an input sequence (the lemma and the target tag)
and to generate an output sequence (the target form).
"""

from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_enc = BertConfig(
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=1024,
    position_embedding_type='absolute',
    hidden_dropout_prob=0.2,
    attention_probs_dropout_prob=0.2,
    vocab_size=len(tokenizer)
)

config_dec = BertConfig(
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=1024,
    position_embedding_type='absolute',
    hidden_dropout_prob=0.2,
    attention_probs_dropout_prob=0.2,
    vocab_size=len(tokenizer),
    is_decoder=True
)


config = EncoderDecoderConfig.from_encoder_decoder_configs(config_enc,config_dec)

model = EncoderDecoderModel(config)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.max_new_tokens = 32
model.generation_config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "attention_probs_dropout_prob": 0.2,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 52
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "add_cross_attention": true,
  "attention_probs_dropout_prob": 0.2,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "interme

In [24]:
tokenizer.bos_token_id

1

In [25]:
#model.model.encoder.embed_positions = BartLearnedPositionalEmbedding(52, 256)
#model.model.decoder.embed_positions = BartLearnedPositionalEmbedding(52, 256)

In [26]:
"""
UTILITIES
"""

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        input, target = self.data[idx]
        return {"input_ids": self.tokenizer(input, padding='longest',
                                            truncation=True,
                                            add_special_tokens=True)['input_ids'],
                "labels": self.tokenizer(target, padding='longest',
                                         truncation=True,
                                         add_special_tokens=True)['input_ids']}

def postprocess_data(token_ids):
    token_ids = np.where(token_ids != -100, token_ids, tokenizer.pad_token_id)
    return tokenizer.batch_decode(token_ids, skip_special_tokens=True)

In [27]:
###DO NOT CHANGE THIS CELL###
"""
MAIN METRIC
"""

import evaluate
import numpy as np
import random

def compute_metrics(eval_preds):
    metric = evaluate.load("exact_match")
    preds, labels = eval_preds
    decoded_preds = postprocess_data(preds)
    decoded_labels = postprocess_data(labels)

    # During development, you can uncomment the lines to see what predictions your model makes
    ks = random.choices(list(range(len(decoded_preds))), k=15)
    print('Predicted:', [decoded_preds[k] for k in ks])
    print('Targets:', [decoded_labels[k] for k in ks])
    print('___________________________________________________________________')

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return result

In [ ]:
"""
TRAINING
"""

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

dataset = {'train': CustomDataset(data['train'], tokenizer),
           'dev': CustomDataset(data['dev'], tokenizer)}
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    max_steps=20000,
    per_device_train_batch_size=400,
    learning_rate = 0.001,
    lr_scheduler_type='inverse_sqrt',
    warmup_steps=4000,
    adam_beta2=0.98,
    label_smoothing_factor=0.1,
    eval_strategy="steps",
    eval_steps=400,
    eval_delay=400,
    save_strategy="steps",
    save_steps=400,
    report_to='none',
    predict_with_generate=True,
    metric_for_best_model='exact_match',
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=400,
    output_dir='baseline_0.2',
    overwrite_output_dir=True,
    load_best_model_at_end=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-28-d66b8fc4d75f>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss,Validation Loss,Exact Match
400,2.559600,1.370481,0.128000
800,1.149300,1.067726,0.404000
1200,0.880900,1.058576,0.436000
1600,0.795200,1.069462,0.451000
2000,0.762200,1.114788,0.468000
2400,0.746800,1.135823,0.491000
2800,0.738200,1.148750,0.482000
3200,0.734000,1.147503,0.496000
3600,0.732000,1.154965,0.474000
4000,0.730800,1.150636,0.500000


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still o

Predicted: ['jiltłʼéééh', 'diiichʼééh', 'ajiłʼá', 'adadoołjid', 'dajitʼeesh', 'dadooshdoł', 'jiʼeeł', 'hashkaał', 'ndaałjoł', 'danichʼééh', 'adajiiłjid', 'jichaaʼ', 'diichʼééh', 'adadiilmáás', 'yisʼáá']
Targets: ['jooltłʼá', 'diichʼééh', 'ajííłʼaʼ', 'adeidoołjił', 'dajitʼeesh', 'yisdádadoołʼoł', 'jiʼeeł', 'haashkaał', 'nidaałjooł', 'diichʼééh', 'adajishjid', 'jíícha', 'diishchʼééh', 'adadiiʼmas', 'yishʼá']
___________________________________________________________________


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


...

Predicted: ['niséniiʼ', 'ílzhih', 'neilé', 'yiisgis', 'yiyiiłtłʼoh', 'yiitłoh', 'nizhdootih', 'yiyíítʼéézh', 'yighaas', 'wołdéél', 'adadoołjił', 'adadiiljił', 'nijishjool', 'diichʼééh', 'nidaʼsiidzo']
Targets: ['niséniiʼ', 'ílizh', 'neilé', 'yiisgis', 'yiyiiłtʼoh', 'yiitłoh', 'nizhdootį́į́ł', 'yizhtʼéézh', 'yighaas', 'woołdéél', 'adadoołjił', 'adadiiljił', 'nijishjool', 'diichʼééh', 'nidaʼsiidzo']
___________________________________________________________________


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Predicted: ['yidoosdił', 'nijitin', 'dahojisghal', 'jinitłʼah', 'hozhdoolneeł', 'jíldin', 'diishchʼééh', 'ndazhdoonih', 'daahkʼá', 'dadoohchʼééh', 'diiltłʼééł', 'hadaooniiʼ', 'nítłʼah', 'dinishjol', 'nisiiʼniiʼ']
Targets: ['yisdéidoołʼoł', 'nijitin', 'dajiisghal', 'jinitłʼah', 'hozhdoolnih', 'jildin', 'diishchʼééh', 'ndazhdoonih', 'daahkʼá', 'dadoohchʼééh', 'diiltłʼééł', 'dahohniih', 'nítłʼah', 'dinishjool', 'nisiiʼniiʼ']
___________________________________________________________________


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Predicted: ['ajiłjiid', 'naaso', 'jiigis', 'nisiiʼnih', 'yiigis', 'dajiiłtsóód', 'adeiilʼá', 'hadajishgeezh', 'oolzhih', 'deiigaas', 'jółtʼaʼ', 'ndaahłé', 'daoołʼaaʼ', 'dayiiłgąsh', 'woołʼaaʼ']
Targets: ['ajiłjiid', 'naʼaso', 'jiigis', 'nisiiʼniiʼ', 'yiigis', 'dajiiłtsóód', 'adeiilʼá', 'hadajishgizh', 'azhlizh', 'deiigaas', 'jółtaʼ', 'nidaahłé', 'daałʼá', 'dayiiłgąsh', 'wołʼá']
___________________________________________________________________


In [ ]:
###DO NOT CHANGE THIS CELL###
"""
EVALUATION
"""

test_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.tst'
test_data = download_data(test_data_path)
test_dataset = CustomDataset(preprocess_data(test_data), tokenizer)
result = trainer.evaluate(test_dataset)
test_accuracy_wu = result['eval_exact_match']



NameError: name 'download_data' is not defined

### Test Accuracy

In [ ]:
###DO NOT CHANGE THIS CELL###
print('Test accuracy:', test_accuracy_wu)

Test accuracy: 0.488


### Report
[Describe your approach in up to 300 words here.]

# Task 2
Build a better morphological inflector.

In [ ]:
# your code goes here - make sure the final test accuracy is saved as test_accuracy

### Test Accuracy

In [ ]:
###DO NOT CHANGE THIS CELL###
print('Test accuracy:', test_accuracy)

### Report
[Describe your approach in up to 300 words here.]